In [0]:
import os
import tensorflow as tf
import numpy as np
import cv2
from google.colab import drive
import matplotlib.pyplot as plt
import json
%matplotlib inline

drive.mount('/content/gdrive')

tf.enable_eager_execution()

layers = tf.keras.layers

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
SCALE = 32
GRID_W, GRID_H = 18, 10
N_CLASSES = 80
N_ANCHORS = 5
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH = GRID_H*SCALE, GRID_W*SCALE, 3

LAMBDA_COORD = 5.0
LAMBDA_NO_OBJ = 0.5
BATCH_SIZE = 10
LEARNING_RATE = 0.0001
NUM_ITERS = 100000
TFRECORD_PATH = './data.tfrecord'
MODEL_PATH = '/model' 
SAVE_INTERVAL = './model', 10000

bn_epsilon = 1e-3

In [0]:
IMAGE_HEIGHT, IMAGE_WIDTH

(320, 576)

In [0]:
def space_to_depth_x2(x):
  return tf.space_to_depth(x, block_size=2)
class yolov2(tf.keras.Model):
  def __init__(self, device='cpu:0', num_anchors=5, num_classes = 80):
    super(yolov2, self).__init__()
    self.device=device
    self.num_anchors = num_anchors
    self.num_classes = num_classes
    
    
    self.conv1 = layers.Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)
    self.bn1 = layers.BatchNormalization(name='norm_1')
    self.lrelu1 = layers.LeakyReLU(alpha=0.1)
    self.maxpool1 = layers.MaxPooling2D(pool_size=(2, 2))
    
    self.conv2 = layers.Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)
    self.bn2 = layers.BatchNormalization(name='norm_2')
    self.lrelu2 = layers.LeakyReLU(alpha=0.1)
    self.maxpool2 = layers.MaxPooling2D(pool_size=(2, 2))
    
    self.conv3 = layers.Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)
    self.bn3 = layers.BatchNormalization(name='norm_3')
    self.lrelu3 = layers.LeakyReLU(alpha=0.1)
    
    self.conv4 = layers.Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)
    self.bn4 = layers.BatchNormalization(name='norm_4')
    self.lrelu4 = layers.LeakyReLU(alpha=0.1)
    
    self.conv5 = layers.Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)
    self.bn5 = layers.BatchNormalization(name='norm_5')
    self.lrelu5 = layers.LeakyReLU(alpha=0.1)
    self.maxpool5 = layers.MaxPooling2D(pool_size=(2, 2))
    
    self.conv6 = layers.Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)
    self.bn6 = layers.BatchNormalization(name='norm_6')
    self.lrelu6 = layers.LeakyReLU(alpha=0.1)
    
    self.conv7 = layers.Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)
    self.bn7 = layers.BatchNormalization(name='norm_7')
    self.lrelu7 = layers.LeakyReLU(alpha=0.1)
    
    self.conv8 = layers.Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)
    self.bn8 = layers.BatchNormalization(name='norm_8')
    self.lrelu8 = layers.LeakyReLU(alpha=0.1)
    self.maxpool8 = layers.MaxPooling2D(pool_size=(2, 2))
    
    self.conv9 = layers.Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)
    self.bn9 = layers.BatchNormalization(name='norm_9')
    self.lrelu9 = layers.LeakyReLU(alpha=0.1)
    
    self.conv10 = layers.Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)
    self.bn10 = layers.BatchNormalization(name='norm_10')
    self.lrelu10 = layers.LeakyReLU(alpha=0.1)
    
    self.conv11 = layers.Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)
    self.bn11 = layers.BatchNormalization(name='norm_11')
    self.lrelu11 = layers.LeakyReLU(alpha=0.1)
    
    self.conv12 = layers.Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)
    self.bn12 = layers.BatchNormalization(name='norm_12')
    self.lrelu12 = layers.LeakyReLU(alpha=0.1)
    
    self.conv13 = layers.Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)
    self.bn13 = layers.BatchNormalization(name='norm_13')
    self.lrelu13 = layers.LeakyReLU(alpha=0.1)
    self.maxpool13 = layers.MaxPooling2D(pool_size=(2, 2))
    
    self.conv14 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)
    self.bn14 = layers.BatchNormalization(name='norm_14')
    self.lrelu14 = layers.LeakyReLU(alpha=0.1)
    
    self.conv15 = layers.Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)
    self.bn15 = layers.BatchNormalization(name='norm_15')
    self.lrelu15 = layers.LeakyReLU(alpha=0.1)
    
    self.conv16 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)
    self.bn16 = layers.BatchNormalization(name='norm_16')
    self.lrelu16 = layers.LeakyReLU(alpha=0.1)
    
    self.conv17 = layers.Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)
    self.bn17 = layers.BatchNormalization(name='norm_17')
    self.lrelu17 = layers.LeakyReLU(alpha=0.1)
    
    self.conv18 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)
    self.bn18 = layers.BatchNormalization(name='norm_18')
    self.lrelu18 = layers.LeakyReLU(alpha=0.1)
    
    self.conv19 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)
    self.bn19 = layers.BatchNormalization(name='norm_19')
    self.lrelu19 = layers.LeakyReLU(alpha=0.1)
    
    self.conv20 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)
    self.bn20 = layers.BatchNormalization(name='norm_20')
    self.lrelu20 = layers.LeakyReLU(alpha=0.1)
    
    self.conv21 = layers.Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)
    self.bn21 = layers.BatchNormalization(name='norm_21')
    self.lrelu21 = layers.LeakyReLU(alpha=0.1)
    self.lamb21 = layers.Lambda(space_to_depth_x2)
    
    self.conv22 = layers.Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)
    self.bn22 = layers.BatchNormalization(name='norm_22')
    self.lrelu22 = layers.LeakyReLU(alpha=0.1)
    
    
    self.conv23 = layers.Conv2D(self.num_anchors*(5+self.num_classes), (1,1), strides = (1,1), padding='same', name='conv_23', use_bias=True)
#     self.bn23 = layers.BatchNormalization(name='norm_23')
    
  def call(self, input, training=True):
    x = self.conv1(input)
    x = self.bn1(x, training=training)
    x = self.lrelu1(x)
    x = self.maxpool1(x)
    
    x = self.conv2(x)
    x = self.bn2(x, training=training)
    x = self.lrelu2(x)
    x = self.maxpool2(x)
    
    x = self.conv3(x)
    x = self.bn3(x, training=training)
    x = self.lrelu3(x)
    
    x = self.conv4(x)
    x = self.bn4(x, training=training)
    x = self.lrelu4(x)
    
    x = self.conv5(x)
    x = self.bn5(x, training=training)
    x = self.lrelu5(x)
    x = self.maxpool5(x)
    
    x = self.conv6(x)
    x = self.bn6(x, training=training)
    x = self.lrelu6(x)
    
    x = self.conv7(x)
    x = self.bn7(x, training=training)
    x = self.lrelu7(x)
    
    x = self.conv8(x)
    x = self.bn8(x, training=training)
    x = self.lrelu8(x)
    x = self.maxpool8(x)
    
    x = self.conv9(x)
    x = self.bn9(x, training=training)
    x = self.lrelu9(x)
    
    x = self.conv10(x)
    x = self.bn10(x, training=training)
    x = self.lrelu10(x)
    
    x = self.conv11(x)
    x = self.bn11(x, training=training)
    x = self.lrelu11(x)
    
    x = self.conv12(x)
    x = self.bn12(x, training=training)
    x = self.lrelu12(x)
    
    x = self.conv13(x)
    x = self.bn13(x, training=training)
    x = self.lrelu13(x)
    
    skip_connection = x
    
    x = self.maxpool13(x)
    
    x = self.conv14(x)
    x = self.bn14(x, training=training)
    x = self.lrelu14(x)
    
    x = self.conv15(x)
    x = self.bn15(x, training=training)
    x = self.lrelu15(x)
    
    x = self.conv16(x)
    x = self.bn16(x, training=training)
    x = self.lrelu16(x)
    
    x = self.conv17(x)
    x = self.bn17(x, training=training)
    x = self.lrelu17(x)
    
    x = self.conv18(x)
    x = self.bn18(x, training=training)
    x = self.lrelu18(x)
    
    x = self.conv19(x)
    x = self.bn19(x, training=training)
    x = self.lrelu19(x)
    
    x = self.conv20(x)
    x = self.bn20(x, training=training)
    x = self.lrelu20(x)
    
    skip_connection = self.conv21(skip_connection)
    skip_connection = self.bn21(skip_connection, training = training)
    skip_connection = self.lrelu21(skip_connection)
    skip_connection = self.lamb21(skip_connection)
    
#     print (skip_connection.shape, x.shape)
    x = layers.concatenate([skip_connection, x])
#     print (x.shape)
    
    x = self.conv22(x)
    x = self.bn22(x, training=training)
    x = self.lrelu22(x)
    
#     print (x.shape)
    self.grid_h = x.shape[1]
    self.grid_w = x.shape[2]
    
    x = self.conv23(x)
#     x = self.bn23(x, training=training)
    y = layers.Reshape((self.grid_h, self.grid_w, self.num_anchors, 5+self.num_classes))(x)
    
    return y

In [0]:
weights_path = '/content/gdrive/My Drive/yolov2.weights'

In [0]:
weight_array = np.fromfile(weights_path, dtype='f')

In [0]:
weight_array[4:]

array([-11.127332  ,   0.13706052,   0.6848585 , ...,  -0.03456368,
         0.01218708,   0.01465192], dtype=float32)

In [0]:
def load_conv_layer_bn(name,loaded_weights,shape,offset):
    # Conv layer with Batch norm

    n_kernel_weights = shape[0]*shape[1]*shape[2]*shape[3]
    n_output_channels = shape[-1]
    n_bn_mean = n_output_channels
    n_bn_var = n_output_channels
    n_biases = n_output_channels
    n_bn_gamma = n_output_channels

    n_weights_conv_bn = (n_kernel_weights + n_output_channels * 4)

    # IMPORTANT: This is where (n_kernel_weights + n_output_channels * 4) comes from: 
    # n_params = kernel_shape + n_biases + n_bn_means + n_bn_var + n_bn_gammas
    # n_params = kernel_shape + n_biases + n_output_channels + n_output_channels + n_output_channels
    # n_params = kernel_shape + n_output_channels + n_output_channels + n_output_channels + n_output_channels
    # n_params = kernel_shape + n_output_channels*4
    # IMPORTANT: YOLOv2 sets the biases in every convolution = 0 and keeps only the betas (offsets) of the Batch Normalization!
    # So in the end there will be only mean,var,beta(offset),gamma(scale) for every single output channel!

    print('Loading '+str(n_weights_conv_bn)+' weights of '+name+' ...')

    biases = loaded_weights[offset:offset+n_biases]
    offset = offset + n_biases
    gammas = loaded_weights[offset:offset+n_bn_gamma]
    offset = offset + n_bn_gamma
    means = loaded_weights[offset:offset+n_bn_mean]
    offset = offset + n_bn_mean
    var = loaded_weights[offset:offset+n_bn_var]
    offset = offset + n_bn_var
    kernel_weights = loaded_weights[offset:offset+n_kernel_weights]
    offset = offset + n_kernel_weights

    # IMPORTANT: DarkNet conv_weights are serialized Caffe-style: (out_dim, in_dim, height, width)
    kernel_weights = np.reshape(kernel_weights,(shape[3],shape[2],shape[0],shape[1]),order='C')

    # IMPORTANT: Denormalize the weights with the Batch Normalization parameters
#     for i in range(n_output_channels):

#         scale = gammas[i] / np.sqrt(var[i] + bn_epsilon)
#         kernel_weights[i,:,:,:] = kernel_weights[i,:,:,:] * scale
#         biases[i] = biases[i] - means[i] * scale

    # IMPORTANT: Set weights to Tensorflow order: (height, width, in_dim, out_dim)
    kernel_weights = np.transpose(kernel_weights,[2,3,1,0])

    return biases,gammas, means, var, kernel_weights,offset

In [0]:
def load_conv_layer_bias(name,loaded_weights,shape,offset):
    # Conv layer with Batch norm

    n_kernel_weights = shape[0]*shape[1]*shape[2]*shape[3]
    n_output_channels = shape[-1]
    n_biases = n_output_channels

    n_weights_conv_bn = (n_kernel_weights + n_output_channels * 4)

    # IMPORTANT: This is where (n_kernel_weights + n_output_channels * 4) comes from: 
    # n_params = kernel_shape + n_biases + n_bn_means + n_bn_var + n_bn_gammas
    # n_params = kernel_shape + n_biases + n_output_channels + n_output_channels + n_output_channels
    # n_params = kernel_shape + n_output_channels + n_output_channels + n_output_channels + n_output_channels
    # n_params = kernel_shape + n_output_channels*4
    # IMPORTANT: YOLOv2 sets the biases in every convolution = 0 and keeps only the betas (offsets) of the Batch Normalization!
    # So in the end there will be only mean,var,beta(offset),gamma(scale) for every single output channel!

    print('Loading '+str(n_weights_conv_bn)+' weights of '+name+' ...')

    biases = loaded_weights[offset:offset+n_biases]
    offset = offset + n_biases
    kernel_weights = loaded_weights[offset:offset+n_kernel_weights]
    offset = offset + n_kernel_weights

    # IMPORTANT: DarkNet conv_weights are serialized Caffe-style: (out_dim, in_dim, height, width)
    kernel_weights = np.reshape(kernel_weights,(shape[3],shape[2],shape[0],shape[1]),order='C')

    # IMPORTANT: Denormalize the weights with the Batch Normalization parameters
#     for i in range(n_output_channels):

#         scale = gammas[i] / np.sqrt(var[i] + bn_epsilon)
#         kernel_weights[i,:,:,:] = kernel_weights[i,:,:,:] * scale
#         biases[i] = biases[i] - means[i] * scale

    # IMPORTANT: Set weights to Tensorflow order: (height, width, in_dim, out_dim)
    kernel_weights = np.transpose(kernel_weights,[2,3,1,0])

    return biases,kernel_weights,offset

In [0]:
weight_array = weight_array[4:]


In [0]:
print('Total number of params to load = {}'.format(len(weight_array)))


Total number of params to load = 50983561


In [0]:
model = yolov2()

def _parse_function(example_proto):

    features={'image': tf.FixedLenFeature([], tf.string),'label': tf.FixedLenFeature([], tf.string)}

    parsed_features = tf.parse_single_example(example_proto, features)
    
    image_raw = tf.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.cast(tf.reshape(image_raw, [IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH]), tf.float32)
    image = image/255.0
#     print (image)
    
    label_raw = tf.decode_raw(parsed_features['label'], tf.float32)	
    label = tf.reshape(label_raw, [GRID_H, GRID_W, N_ANCHORS, 6])

    return image, label
  
  
val_dataset = tf.data.TFRecordDataset('/content/gdrive/My Drive/cocostuff2017/val2017.tfrecord')
val_dataset = val_dataset.map(_parse_function)
val_dataset = val_dataset.shuffle(buffer_size=100)
val_dataset = val_dataset.batch(1)

for img in val_dataset:
#   print (img)
  plt.imshow(img[0][0].numpy()[:,:,::-1])
  plt.grid(b=None)
  break
  
model(img[0])

In [0]:
for v in model.trainable_variables:
  print (v.shape)

(3, 3, 3, 32)
(32,)
(32,)
(3, 3, 32, 64)
(64,)
(64,)
(3, 3, 64, 128)
(128,)
(128,)
(1, 1, 128, 64)
(64,)
(64,)
(3, 3, 64, 128)
(128,)
(128,)
(3, 3, 128, 256)
(256,)
(256,)
(1, 1, 256, 128)
(128,)
(128,)
(3, 3, 128, 256)
(256,)
(256,)
(3, 3, 256, 512)
(512,)
(512,)
(1, 1, 512, 256)
(256,)
(256,)
(3, 3, 256, 512)
(512,)
(512,)
(1, 1, 512, 256)
(256,)
(256,)
(3, 3, 256, 512)
(512,)
(512,)
(3, 3, 512, 1024)
(1024,)
(1024,)
(1, 1, 1024, 512)
(512,)
(512,)
(3, 3, 512, 1024)
(1024,)
(1024,)
(1, 1, 1024, 512)
(512,)
(512,)
(3, 3, 512, 1024)
(1024,)
(1024,)
(3, 3, 1024, 1024)
(1024,)
(1024,)
(3, 3, 1024, 1024)
(1024,)
(1024,)
(1, 1, 512, 64)
(64,)
(64,)
(3, 3, 1280, 1024)
(1024,)
(1024,)
(1, 1, 1024, 425)
(425,)


In [0]:
offset = 0
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv1',weight_array,[3,3,3,32],offset)
model.conv1.weights[0].assign(kernel_weights)
model.bn1.beta.assign(biases)
model.bn1.gamma.assign(gammas)
model.bn1.moving_mean.assign(means)
model.bn1.moving_variance.assign(var)

Loading 992 weights of conv1 ...


<tf.Variable 'UnreadVariable' shape=(32,) dtype=float32, numpy=
array([0.1267794 , 0.02473054, 0.11996339, 0.01351726, 0.20216261,
       0.00812943, 0.16426097, 0.03903044, 0.19388086, 0.14270157,
       0.01206353, 0.12574638, 0.2808188 , 0.09807997, 0.00530833,
       0.00247807, 0.30808368, 0.07031066, 0.05335343, 0.18778479,
       0.03780013, 0.1485945 , 0.0342221 , 0.14622688, 0.02972092,
       0.00340325, 0.26288828, 0.17489278, 0.08435263, 0.05135241,
       0.08448725, 0.04981071], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv2',weight_array,[3,3,32,64],offset)
model.conv2.weights[0].assign(kernel_weights)
model.bn2.beta.assign(biases)
model.bn2.gamma.assign(gammas)
model.bn2.moving_mean.assign(means)
model.bn2.moving_variance.assign(var)

Loading 18688 weights of conv2 ...


<tf.Variable 'UnreadVariable' shape=(64,) dtype=float32, numpy=
array([ 4.0619597,  7.3567696,  4.857758 ,  8.371428 ,  8.161284 ,
        7.3098674,  6.949231 ,  2.912455 ,  8.948019 ,  5.236448 ,
        7.285501 ,  9.341274 ,  3.8692305,  3.8976314,  8.912858 ,
        9.706576 ,  7.97693  ,  3.7434084, 13.057736 ,  3.9516516,
       10.272068 ,  7.7368064,  8.928091 , 14.273567 ,  3.5122283,
        5.7471647, 12.57585  ,  4.3725843, 24.780958 , 13.555199 ,
        6.306083 , 11.185461 ,  5.9309096, 10.961324 ,  4.1662016,
       15.897675 ,  8.0033865,  7.343186 , 27.747704 , 11.147099 ,
        4.296344 ,  7.6500998, 11.488465 ,  8.575572 , 34.039303 ,
        1.7889409, 14.830605 , 15.552972 , 10.137439 ,  3.304091 ,
       13.185245 ,  6.9953113,  6.8802986, 10.001974 , 10.010253 ,
       10.670198 ,  4.5359087, 12.880255 ,  4.264838 , 17.226027 ,
        8.8331785,  5.9292154,  4.010449 ,  2.990743 ], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv3',weight_array,[3,3,64,128],offset)
model.conv3.weights[0].assign(kernel_weights)
model.bn3.beta.assign(biases)
model.bn3.gamma.assign(gammas)
model.bn3.moving_mean.assign(means)
model.bn3.moving_variance.assign(var)

Loading 74240 weights of conv3 ...


<tf.Variable 'UnreadVariable' shape=(128,) dtype=float32, numpy=
array([12.247726 , 12.315641 , 10.871019 ,  4.213533 ,  4.6180468,
        4.227156 ,  5.2461815, 15.303814 ,  6.9430757,  3.822708 ,
        4.395547 ,  3.001818 ,  7.212768 ,  3.8371553,  7.7466984,
        3.3768604, 14.217424 ,  5.327592 ,  5.493556 ,  2.4996636,
        3.8318088, 13.441071 ,  7.147267 ,  8.4155035,  7.424852 ,
        5.5471153, 15.391259 , 15.274406 ,  4.5877476, 10.940833 ,
        8.091756 ,  5.2940426,  6.493051 ,  3.8197656, 10.213662 ,
       11.403578 ,  4.21481  ,  3.0315216,  6.728792 ,  7.177107 ,
        8.659136 ,  5.258941 ,  5.2698245,  8.610996 ,  7.0605655,
        5.9963026, 11.304926 , 10.698402 ,  7.856121 ,  8.014207 ,
        8.712148 ,  5.577378 , 10.480687 ,  7.977638 ,  4.4585123,
        9.996297 ,  5.5089602,  5.468943 ,  5.002474 ,  6.2079053,
        2.3294435,  4.5226893,  5.528904 ,  3.9498818, 12.213737 ,
        4.1863217, 17.622276 ,  8.316085 ,  6.339471 ,  8.874878

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv4',weight_array,[1,1,128,64],offset)
model.conv4.weights[0].assign(kernel_weights)
model.bn4.beta.assign(biases)
model.bn4.gamma.assign(gammas)
model.bn4.moving_mean.assign(means)
model.bn4.moving_variance.assign(var)

Loading 8448 weights of conv4 ...


<tf.Variable 'UnreadVariable' shape=(64,) dtype=float32, numpy=
array([0.90442115, 1.722256  , 1.1964574 , 1.6960069 , 1.6499394 ,
       0.93029773, 0.9361636 , 1.3862314 , 1.7923322 , 1.1911128 ,
       1.9937105 , 1.1114237 , 1.7881083 , 1.3851993 , 2.6650908 ,
       2.5282936 , 2.1132386 , 1.7615217 , 1.0739288 , 0.5666174 ,
       1.8861904 , 1.9852065 , 2.6935022 , 0.632474  , 2.2361763 ,
       1.934308  , 1.1536447 , 1.244238  , 1.3616742 , 1.6745968 ,
       1.6100582 , 2.0669549 , 4.5564446 , 1.0442138 , 1.4802948 ,
       1.9442935 , 2.995374  , 1.465055  , 1.3719602 , 0.7301104 ,
       3.553423  , 0.8447595 , 0.29234713, 1.3423241 , 1.1904811 ,
       2.435231  , 0.94794583, 1.0399876 , 1.7541808 , 1.830843  ,
       1.0541443 , 1.4609584 , 0.8983601 , 1.988832  , 3.1493752 ,
       1.1987455 , 1.2517608 , 1.4212646 , 1.9138681 , 1.3478317 ,
       1.8122778 , 1.5451376 , 2.3102622 , 1.8814226 ], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv5',weight_array,[3,3,64,128],offset)
model.conv5.weights[0].assign(kernel_weights)
model.bn5.beta.assign(biases)
model.bn5.gamma.assign(gammas)
model.bn5.moving_mean.assign(means)
model.bn5.moving_variance.assign(var)

Loading 74240 weights of conv5 ...


<tf.Variable 'UnreadVariable' shape=(128,) dtype=float32, numpy=
array([ 8.753701 , 14.734301 ,  5.7672863,  9.589363 ,  6.964862 ,
        8.801    , 12.852385 ,  8.350551 , 10.243819 ,  9.430542 ,
       13.547287 , 12.425217 ,  7.7991915, 12.868789 ,  6.7491975,
        9.066273 ,  6.431867 , 12.087756 , 15.990185 , 13.830545 ,
       12.132839 , 12.705175 , 10.589825 ,  7.2138553, 20.782763 ,
        9.231889 , 10.187213 , 11.357267 , 12.055616 ,  9.585322 ,
       11.685738 ,  9.509375 , 10.476166 ,  7.6955924,  8.208106 ,
        8.540763 ,  7.53484  ,  9.763281 ,  9.76626  ,  6.2514176,
        8.645658 , 13.320235 , 11.17949  ,  7.4095974,  9.1364   ,
        8.956989 , 15.555501 ,  9.294776 ,  8.693219 , 11.651982 ,
       11.554038 ,  9.751885 , 10.360676 , 11.801446 ,  9.780753 ,
        9.142796 , 11.182859 ,  5.725019 , 12.089443 ,  8.931024 ,
       12.56432  ,  6.8891754, 24.610483 ,  8.759512 ,  8.683462 ,
        9.900074 ,  8.250755 , 10.5721855, 13.594646 ,  8.765009

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv6',weight_array,[3,3,128,256],offset)
model.conv6.weights[0].assign(kernel_weights)
model.bn6.beta.assign(biases)
model.bn6.gamma.assign(gammas)
model.bn6.moving_mean.assign(means)
model.bn6.moving_variance.assign(var)

Loading 295936 weights of conv6 ...


<tf.Variable 'UnreadVariable' shape=(256,) dtype=float32, numpy=
array([ 6.5663695,  4.334992 ,  7.589168 ,  9.035857 ,  5.895871 ,
        4.2153945,  6.1218143,  3.4631395,  9.222993 ,  5.156825 ,
        7.056094 ,  5.7759147,  4.3133736,  3.7605228,  3.852077 ,
        7.111323 ,  4.1558247,  6.3070397,  3.1286619,  6.0274477,
        6.1770673,  2.8362768,  5.1536884,  3.2348125,  5.0753355,
        4.612634 ,  3.9916067, 13.947053 ,  3.8729384,  9.4645   ,
        3.2163608,  3.369157 ,  4.86494  ,  2.6977942, 15.386446 ,
        3.729473 ,  2.5155632,  4.7232842,  8.953746 ,  8.56003  ,
        4.5695233,  4.3212667,  3.251783 ,  5.0039263,  2.9397237,
        5.861437 ,  9.8877945,  5.792236 ,  5.3124285,  3.3046844,
        4.265853 ,  7.330517 ,  5.601845 ,  6.4636025,  4.347238 ,
        4.9356847,  7.033282 ,  3.7686582,  6.139436 ,  5.7245126,
        5.321247 ,  6.0347724,  2.2446382,  7.124702 ,  5.5816483,
        8.511965 ,  5.971804 ,  4.236525 ,  3.5527275,  7.540963

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv7',weight_array,[1,1,256,128],offset)
model.conv7.weights[0].assign(kernel_weights)
model.bn7.beta.assign(biases)
model.bn7.gamma.assign(gammas)
model.bn7.moving_mean.assign(means)
model.bn7.moving_variance.assign(var)

Loading 33280 weights of conv7 ...


<tf.Variable 'UnreadVariable' shape=(128,) dtype=float32, numpy=
array([0.81795835, 1.4389377 , 0.7180596 , 0.9022191 , 0.9565567 ,
       1.2237482 , 0.8040891 , 0.8936103 , 0.52009845, 0.8441915 ,
       3.513131  , 3.3201473 , 1.413556  , 0.7009343 , 0.8451079 ,
       1.4957908 , 0.45027885, 0.59907675, 0.5577496 , 2.0553186 ,
       0.94655657, 0.8396839 , 1.0551165 , 0.57779545, 0.98765266,
       0.8997892 , 0.9327548 , 0.923499  , 1.3354025 , 1.5870802 ,
       1.2083591 , 0.59714425, 0.90033305, 0.8938521 , 1.1177701 ,
       2.0417192 , 0.9830701 , 1.2068077 , 1.3841232 , 1.2883826 ,
       0.73242253, 1.7639621 , 0.9746352 , 1.1397756 , 0.90826267,
       0.825645  , 2.1213486 , 2.9334538 , 0.9242777 , 0.9195833 ,
       0.8403086 , 3.1329513 , 0.6415645 , 0.90794075, 0.61945087,
       1.1901933 , 1.3872138 , 2.0521016 , 1.2919072 , 0.7932459 ,
       1.6170293 , 1.9821637 , 0.8108735 , 1.2511247 , 0.6308862 ,
       1.2885554 , 0.93914133, 0.536718  , 0.93712133, 1.5145973

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv8',weight_array,[3,3,128,256],offset)
model.conv8.weights[0].assign(kernel_weights)
model.bn8.beta.assign(biases)
model.bn8.gamma.assign(gammas)
model.bn8.moving_mean.assign(means)
model.bn8.moving_variance.assign(var)

Loading 295936 weights of conv8 ...


<tf.Variable 'UnreadVariable' shape=(256,) dtype=float32, numpy=
array([ 5.618415 ,  6.7643056,  7.746937 ,  7.5826387, 11.823571 ,
        5.365489 ,  5.246611 ,  7.9445086,  7.50529  ,  5.141972 ,
        5.3517632,  5.2179103,  6.252713 ,  5.6116858,  5.9981155,
        6.49978  ,  6.4543686,  4.5796638,  8.156799 ,  7.8101416,
        6.04217  ,  9.552836 ,  7.6356893,  5.8406763,  8.88329  ,
        7.1488323,  5.401253 ,  4.256576 , 11.546443 ,  4.9355664,
        5.3662877,  5.692864 ,  4.791883 ,  4.820044 ,  5.9902806,
        5.3596544,  5.4266267,  6.548955 ,  6.5582967,  4.816807 ,
        7.2108803,  5.8079963,  5.584157 ,  5.4901624,  7.0938544,
        7.716312 ,  4.5753965,  4.02486  ,  8.881737 ,  7.667692 ,
        6.2208714,  5.8543696,  4.5351267,  6.417763 ,  5.0378246,
        5.889655 ,  5.40209  ,  6.7217894,  7.550645 ,  8.525421 ,
        8.8612   ,  5.8865232,  8.761043 ,  5.36472  ,  8.284375 ,
        4.901676 ,  6.4594226, 10.869501 ,  3.8912234,  7.475740

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv9',weight_array,[3,3,256,512],offset)
model.conv9.weights[0].assign(kernel_weights)
model.bn9.beta.assign(biases)
model.bn9.gamma.assign(gammas)
model.bn9.moving_mean.assign(means)
model.bn9.moving_variance.assign(var)

Loading 1181696 weights of conv9 ...


<tf.Variable 'UnreadVariable' shape=(512,) dtype=float32, numpy=
array([2.428497 , 1.7477009, 2.8601456, 6.3240676, 2.6262076, 2.1759036,
       2.7096603, 2.8099537, 2.8212159, 3.237586 , 2.9105766, 2.973522 ,
       3.9149654, 1.7977502, 3.020744 , 1.6887097, 1.9554939, 1.8154855,
       3.0291061, 1.5898602, 4.2401595, 4.115473 , 1.5427148, 4.7633038,
       4.5463996, 3.4081964, 2.5705168, 1.725334 , 3.604553 , 3.9314668,
       2.4503598, 2.3491406, 4.4405527, 5.958826 , 2.4958537, 3.386489 ,
       3.2368922, 2.0923605, 1.5247612, 2.009341 , 2.867646 , 2.3267946,
       3.045943 , 2.3602083, 3.0685067, 2.540547 , 3.2429228, 2.5284286,
       2.3057497, 2.2025833, 1.9431027, 2.27217  , 3.2399483, 2.8996866,
       1.9744202, 3.1122353, 2.2505467, 2.4525764, 3.2999263, 1.7285377,
       1.7535236, 2.113727 , 2.5684206, 5.7958813, 1.9502797, 2.5236218,
       1.6653004, 2.3541946, 2.2589202, 2.3632746, 2.8662367, 3.1432307,
       2.7387419, 3.3797786, 1.5460572, 2.5195165, 2.181355

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv10',weight_array,[1,1,512,256],offset)
model.conv10.weights[0].assign(kernel_weights)
model.bn10.beta.assign(biases)
model.bn10.gamma.assign(gammas)
model.bn10.moving_mean.assign(means)
model.bn10.moving_variance.assign(var)

Loading 132096 weights of conv10 ...


<tf.Variable 'UnreadVariable' shape=(256,) dtype=float32, numpy=
array([0.41510162, 0.63463414, 0.5322651 , 0.21479475, 0.37335736,
       0.43211368, 0.40238234, 0.33164403, 0.53078544, 0.4147117 ,
       0.44734251, 0.26608855, 0.7336681 , 0.3313443 , 0.46101835,
       0.38111302, 1.2114451 , 0.47077087, 0.3335264 , 0.5285532 ,
       0.228299  , 0.47102204, 0.5414285 , 0.5076673 , 0.43119133,
       0.49901626, 0.36860204, 0.27138963, 0.29742488, 0.2565381 ,
       0.34688056, 0.9244518 , 0.3764071 , 0.42520058, 0.36553547,
       0.8971308 , 1.0424433 , 0.26466972, 0.3343132 , 0.79965734,
       0.7397273 , 0.43840638, 0.62168723, 0.3423347 , 0.3784856 ,
       1.5772389 , 0.5474222 , 0.31977662, 0.3328704 , 0.32483372,
       0.7114788 , 0.342586  , 0.45476896, 0.5600881 , 0.28445962,
       0.5429378 , 0.36109474, 0.41539785, 0.39174774, 0.54788905,
       1.0912955 , 0.39340723, 0.38129053, 0.3560435 , 0.43577585,
       0.6011902 , 0.34547913, 0.34671658, 0.3440561 , 0.4441799

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv11',weight_array,[3,3,256,512],offset)
model.conv11.weights[0].assign(kernel_weights)
model.bn11.beta.assign(biases)
model.bn11.gamma.assign(gammas)
model.bn11.moving_mean.assign(means)
model.bn11.moving_variance.assign(var)

Loading 1181696 weights of conv11 ...


<tf.Variable 'UnreadVariable' shape=(512,) dtype=float32, numpy=
array([2.6376343, 3.0442746, 1.597823 , 1.3005162, 2.146318 , 1.1663312,
       2.3929088, 1.6330917, 2.9168406, 2.1542764, 3.4323037, 2.4022012,
       2.209891 , 1.4661086, 3.551415 , 3.507348 , 1.8578216, 1.9703653,
       1.5016136, 3.3496046, 4.0897965, 2.5718997, 1.5317094, 3.145838 ,
       2.2230508, 2.9843338, 1.9326872, 1.3023561, 1.9645168, 1.9381057,
       1.2544767, 1.5041702, 2.155824 , 1.8345455, 2.9552565, 2.8816266,
       2.5516703, 2.3310015, 3.4553154, 1.193066 , 1.9770125, 4.199033 ,
       1.629116 , 4.6776648, 2.5647519, 1.5319506, 1.9847661, 1.304303 ,
       1.8243033, 1.4800965, 1.7744612, 3.3631787, 1.8506134, 3.0686526,
       4.232878 , 2.0578861, 1.6367116, 1.9103554, 2.0806398, 2.643146 ,
       1.2894549, 1.4314139, 2.1509116, 3.9080644, 2.1864684, 1.178166 ,
       1.7894425, 2.2085388, 1.6034541, 1.8335298, 2.6576314, 3.464042 ,
       3.4589465, 1.3263122, 3.8523202, 2.6014154, 1.617139

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv12',weight_array,[1,1,512,256],offset)
model.conv12.weights[0].assign(kernel_weights)
model.bn12.beta.assign(biases)
model.bn12.gamma.assign(gammas)
model.bn12.moving_mean.assign(means)
model.bn12.moving_variance.assign(var)

Loading 132096 weights of conv12 ...


<tf.Variable 'UnreadVariable' shape=(256,) dtype=float32, numpy=
array([0.34256476, 0.6019114 , 0.4642611 , 0.52188194, 0.52714574,
       0.52710134, 0.6348681 , 0.7145923 , 0.67773765, 0.55317235,
       0.45398524, 0.528179  , 0.48560867, 0.790744  , 0.7652089 ,
       0.32825217, 0.42433724, 0.45146742, 0.6760999 , 0.4851038 ,
       0.42440984, 0.73438513, 0.92197293, 0.56469244, 0.45548192,
       0.58741224, 0.391778  , 0.49492007, 0.97453177, 0.7812241 ,
       0.78030175, 0.5033174 , 0.4075138 , 0.6541533 , 0.5055661 ,
       0.5851324 , 0.46128574, 0.58518016, 0.55280477, 0.6157183 ,
       0.56793827, 0.5660823 , 0.6111911 , 0.39671278, 0.34754452,
       1.5822028 , 0.4480544 , 0.5741164 , 0.68361104, 0.53034556,
       0.5074722 , 0.81578255, 0.5701235 , 0.37343463, 0.47896096,
       0.520255  , 0.6533122 , 0.38774002, 0.37694895, 0.52056295,
       0.71013635, 0.573523  , 0.43463627, 0.687905  , 0.56473625,
       0.502194  , 0.37714535, 0.67286325, 0.34103304, 0.5203904

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv13',weight_array,[3,3,256,512],offset)
model.conv13.weights[0].assign(kernel_weights)
model.bn13.beta.assign(biases)
model.bn13.gamma.assign(gammas)
model.bn13.moving_mean.assign(means)
model.bn13.moving_variance.assign(var)

Loading 1181696 weights of conv13 ...


<tf.Variable 'UnreadVariable' shape=(512,) dtype=float32, numpy=
array([1.7545485, 2.7821734, 1.7749467, 1.7988496, 3.2099087, 3.453845 ,
       3.731834 , 1.9657023, 2.0148149, 2.13689  , 2.8189254, 2.970882 ,
       2.285873 , 2.466009 , 2.1519437, 2.3049233, 1.907639 , 3.2385733,
       1.7547071, 1.9638338, 2.2837772, 2.4188707, 2.5632215, 2.5683331,
       2.3746006, 1.7023532, 2.0352209, 1.916039 , 2.4221122, 1.9365482,
       2.5095015, 1.637622 , 2.0596986, 2.793818 , 1.6913284, 1.5611026,
       1.6278286, 2.4819605, 2.4887433, 2.3592014, 1.8902261, 1.6954389,
       1.7094647, 2.796274 , 2.5036645, 1.9657637, 2.259586 , 1.7944659,
       2.9305742, 2.18302  , 3.2236645, 2.4395185, 1.4071369, 2.0585394,
       2.237152 , 3.242589 , 1.9309568, 2.0882418, 3.679653 , 1.6830243,
       3.1576138, 2.265776 , 1.9398503, 2.2064705, 1.8691007, 2.1716883,
       1.851492 , 4.0224924, 2.9467459, 3.1372113, 2.184199 , 1.6658039,
       1.9439467, 2.2070153, 2.2715983, 1.6752429, 1.477200

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv14',weight_array,[3,3,512,1024],offset)
model.conv14.weights[0].assign(kernel_weights)
model.bn14.beta.assign(biases)
model.bn14.gamma.assign(gammas)
model.bn14.moving_mean.assign(means)
model.bn14.moving_variance.assign(var)

Loading 4722688 weights of conv14 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([0.8549979, 1.7160109, 1.9317622, ..., 1.2200478, 1.7245911,
       1.27984  ], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv15',weight_array,[1,1,1024,512],offset)
model.conv15.weights[0].assign(kernel_weights)
model.bn15.beta.assign(biases)
model.bn15.gamma.assign(gammas)
model.bn15.moving_mean.assign(means)
model.bn15.moving_variance.assign(var)

Loading 526336 weights of conv15 ...


<tf.Variable 'UnreadVariable' shape=(512,) dtype=float32, numpy=
array([0.19907045, 0.18509527, 0.16406235, 0.20054363, 0.20296524,
       0.3178269 , 0.16401316, 0.21693127, 0.28537014, 0.24933146,
       0.24444468, 0.2741683 , 0.15001763, 0.2591233 , 0.1804155 ,
       0.2392526 , 0.17527583, 0.22651576, 0.2996639 , 0.3750561 ,
       0.16123612, 0.2294285 , 0.24409519, 0.24032089, 0.26709712,
       0.24034455, 0.46198145, 0.2813802 , 0.1667806 , 0.2861582 ,
       0.14482832, 0.5480209 , 0.19584493, 0.1812438 , 0.31857306,
       0.19060883, 0.19888651, 0.17286359, 0.19055662, 0.3132205 ,
       0.20105714, 0.2130681 , 0.29307556, 0.3012558 , 0.25627738,
       0.27605549, 0.20987275, 0.13917905, 0.3445648 , 0.20018777,
       0.23252608, 0.24833858, 0.36720857, 0.3001791 , 0.18616992,
       0.2298668 , 0.23527852, 0.17223296, 0.17332122, 0.3027776 ,
       0.23449883, 0.24353664, 0.15370648, 0.20929846, 0.24195257,
       0.26279965, 0.21554576, 0.16873458, 0.22018692, 0.1869391

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv16',weight_array,[3,3,512,1024],offset)
model.conv16.weights[0].assign(kernel_weights)
model.bn16.beta.assign(biases)
model.bn16.gamma.assign(gammas)
model.bn16.moving_mean.assign(means)
model.bn16.moving_variance.assign(var)

Loading 4722688 weights of conv16 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([0.67514735, 0.59158045, 1.0752332 , ..., 0.7409849 , 0.59585977,
       4.369199  ], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv17',weight_array,[1,1,1024,512],offset)
model.conv17.weights[0].assign(kernel_weights)
model.bn17.beta.assign(biases)
model.bn17.gamma.assign(gammas)
model.bn17.moving_mean.assign(means)
model.bn17.moving_variance.assign(var)

Loading 526336 weights of conv17 ...


<tf.Variable 'UnreadVariable' shape=(512,) dtype=float32, numpy=
array([0.59701455, 0.47502235, 0.4078404 , 0.25926992, 0.25856805,
       0.51542324, 0.39065027, 0.18284397, 0.33674687, 0.58529043,
       0.43468773, 0.23760039, 0.3975957 , 0.23593311, 0.23479135,
       0.5400699 , 0.29271236, 0.46221504, 0.25850996, 0.29648882,
       0.22333838, 0.42605475, 0.20933922, 0.35095868, 0.5590867 ,
       0.6975973 , 0.39485088, 0.33132565, 0.27059487, 0.21974964,
       0.4311727 , 0.23245817, 0.22382043, 0.31202218, 0.507962  ,
       0.36929452, 0.27130902, 0.53035855, 0.33079138, 0.3534851 ,
       0.47554913, 0.34775612, 0.26106527, 0.4257143 , 0.28163007,
       0.6058284 , 0.33708668, 0.49421975, 0.20101395, 0.34774637,
       0.15035734, 0.25112948, 0.62200624, 0.31726763, 0.19535613,
       0.19534014, 0.33106038, 0.4663885 , 0.244601  , 0.22465068,
       0.33264616, 0.37472093, 0.40421516, 0.2518389 , 0.42415175,
       0.36357766, 0.30907   , 0.4093728 , 0.59318525, 0.374619 

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv18',weight_array,[3,3,512,1024],offset)
model.conv18.weights[0].assign(kernel_weights)
model.bn18.beta.assign(biases)
model.bn18.gamma.assign(gammas)
model.bn18.moving_mean.assign(means)
model.bn18.moving_variance.assign(var)

Loading 4722688 weights of conv18 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([0.35195643, 0.5425942 , 0.277944  , ..., 0.4964034 , 0.39173248,
       0.41862336], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv19',weight_array,[3,3,1024,1024],offset)
model.conv19.weights[0].assign(kernel_weights)
model.bn19.beta.assign(biases)
model.bn19.gamma.assign(gammas)
model.bn19.moving_mean.assign(means)
model.bn19.moving_variance.assign(var)

Loading 9441280 weights of conv19 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([15.937646, 20.30293 , 11.940027, ..., 12.288213, 12.059494,
       18.239151], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv20',weight_array,[3,3,1024,1024],offset)
model.conv20.weights[0].assign(kernel_weights)
model.bn20.beta.assign(biases)
model.bn20.gamma.assign(gammas)
model.bn20.moving_mean.assign(means)
model.bn20.moving_variance.assign(var)

Loading 9441280 weights of conv20 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([1.0227262, 2.3319654, 0.8548247, ..., 0.7651879, 0.9662371,
       1.5361518], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv21',weight_array,[1,1,512,64],offset)
model.conv21.weights[0].assign(kernel_weights)
model.bn21.beta.assign(biases)
model.bn21.gamma.assign(gammas)
model.bn21.moving_mean.assign(means)
model.bn21.moving_variance.assign(var)

Loading 33024 weights of conv21 ...


<tf.Variable 'UnreadVariable' shape=(64,) dtype=float32, numpy=
array([0.24212418, 0.33148378, 0.41202483, 0.25958675, 0.21350436,
       0.22509333, 0.2709969 , 0.17020205, 0.44478244, 0.33066607,
       0.55066234, 0.23744266, 0.36641252, 0.18709314, 0.3201306 ,
       0.21480955, 0.21952143, 0.34774113, 0.31400356, 0.19078705,
       0.23741838, 0.25380865, 0.2996414 , 0.35577708, 0.3105349 ,
       0.27270937, 0.3152936 , 0.29013962, 0.35890666, 0.20509198,
       0.2443307 , 0.24176161, 0.2281428 , 0.27277395, 0.54913104,
       0.20762023, 0.21617706, 0.3322807 , 0.20464967, 0.30504832,
       0.20447296, 0.2689519 , 0.48529   , 0.37925184, 0.22547369,
       0.34241062, 0.26591176, 0.32345858, 0.29726693, 0.19383647,
       0.28294396, 0.25074208, 0.22491062, 0.25194317, 0.1864326 ,
       0.23876876, 0.22808188, 0.1918722 , 0.47826347, 0.21437946,
       0.27419055, 0.28049555, 0.21913326, 0.36748725], dtype=float32)>

In [0]:
biases,gammas, means, var, kernel_weights,offset = load_conv_layer_bn('conv22',weight_array,[3,3,1280,1024],offset)
model.conv22.weights[0].assign(kernel_weights)
model.bn22.beta.assign(biases)
model.bn22.gamma.assign(gammas)
model.bn22.moving_mean.assign(means)
model.bn22.moving_variance.assign(var)

Loading 11800576 weights of conv22 ...


<tf.Variable 'UnreadVariable' shape=(1024,) dtype=float32, numpy=
array([ 0.8531995 , 14.925848  ,  1.0134466 , ...,  0.90744734,
        1.9388497 ,  1.6105366 ], dtype=float32)>

In [0]:
biases, kernel_weights,offset = load_conv_layer_bias('conv23',weight_array,[1,1,1024,425],offset)
model.conv23.weights[0].assign(kernel_weights)
model.conv23.bias.assign(biases)

Loading 436900 weights of conv23 ...


<tf.Variable 'UnreadVariable' shape=(425,) dtype=float32, numpy=
array([ 1.75015330e-02,  9.13802534e-03, -1.44301087e-01, -8.59099627e-02,
       -2.76880121e+00,  2.41153049e+00,  1.32972673e-01,  8.46582294e-01,
       -7.61598647e-02, -2.62515634e-01, -2.26821154e-01, -3.02440912e-01,
        7.27827894e-03,  4.96686995e-02,  3.01793553e-02, -2.42600292e-01,
       -1.89709410e-01, -2.68636018e-01,  2.23143548e-01,  4.21896905e-01,
       -1.75361514e-01,  1.05554029e-01, -5.60857505e-02, -1.41756967e-01,
       -1.29984170e-01, -2.56508172e-01, -3.77901345e-01, -3.62451494e-01,
       -3.87659758e-01,  2.53393352e-01,  1.63245052e-01,  5.08511066e-01,
        1.36528790e-01, -6.90212846e-02, -9.99254361e-02, -1.89874887e-01,
       -2.63391525e-01,  2.51981080e-01, -1.20982870e-01, -1.94312006e-01,
       -2.03608543e-01, -1.11454807e-01, -8.43803361e-02, -1.56823739e-01,
        8.10657263e-01,  2.22688280e-02,  7.34472036e-01, -4.62962165e-02,
        1.34656042e-01,  8.88474211

In [0]:
model.conv23.bias.shape

In [0]:
50983561-offset

In [0]:
model.save_weights('./newweight/checkpoint')

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [0]:
!cp -r ./newweight /content/gdrive/My\ Drive/